In [3]:
%pip install -U -q "google-generativeai>=0.8.3"

Note: you may need to restart the kernel to use updated packages.


In [4]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

## API Key Setup

In [10]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key= GOOGLE_API_KEY)

## Using gemini-1.5-flash

Creating model instance to use for generating response.

In [11]:
flash = genai.GenerativeModel('gemini-1.5-flash')

In [12]:
Markdown(flash.generate_content("Explain Bayes Theorem to a 5 year old kid").text)

Imagine you have a box of toys. Some are red, some are blue, and some are green. You want to know how many red toys are in the box.

Now, imagine your friend tells you that she saw a red toy in the box. This is like having some **evidence**!

Bayes Theorem helps us figure out how much more likely it is that a toy is red **now that we know our friend saw a red toy.**

Think of it like this:

* **Before your friend told you**, you knew there were some red toys in the box, but you weren't sure how many.
* **After your friend told you**, it's more likely that a toy you pick is red, because we know there's at least one red toy.

Bayes Theorem helps us calculate **how much more likely** it is that a toy is red **after** we got the new information.

It's like a special formula that helps us update our guess about the toys in the box based on new clues!


## Starting a chatbot


In [13]:
#chat instance to start chat
chat = flash.start_chat(history=[])

# creating response obj to send message to the chat
response = chat.send_message("Hello Gemini!")
Markdown(response.text)

Hello! It's nice to hear from you. What can I do for you today? 


In [14]:
response

#text is a key in response JSON.
#response is in JSON format. You can print other keys as well

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Hello! It's nice to hear from you. What can I do for you today? \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
     

In [15]:
#checking persistance of conversation state.

Markdown(chat.send_message("value of X is 5!").text)

You're saying the value of X is 5!  

That's great!  

Do you have a question about X or are you just telling me its value? I'm ready to help if you need anything else! 


In [16]:
print(chat.send_message("What is the value of X?").usage_metadata)

prompt_token_count: 94
candidates_token_count: 23
total_token_count: 117



## Models in genai instance

In [17]:
i = 1
for model in genai.list_models():
    print(i,')  ',model.display_name)
    print(model.name)
    print(model.description)
    print()
    i +=1
    

1 )   PaLM 2 Chat (Legacy)
models/chat-bison-001
A legacy text-only model optimized for chat conversations

2 )   PaLM 2 (Legacy)
models/text-bison-001
A legacy model that understands text and generates text as an output

3 )   Embedding Gecko
models/embedding-gecko-001
Obtain a distributed representation of a text.

4 )   Gemini 1.0 Pro Latest
models/gemini-1.0-pro-latest
The best model for scaling across a wide range of tasks. This is the latest model.

5 )   Gemini 1.0 Pro
models/gemini-1.0-pro
The best model for scaling across a wide range of tasks

6 )   Gemini 1.0 Pro
models/gemini-pro
The best model for scaling across a wide range of tasks

7 )   Gemini 1.0 Pro 001 (Tuning)
models/gemini-1.0-pro-001
The best model for scaling across a wide range of tasks. This is a stable model that supports tuning.

8 )   Gemini 1.0 Pro Vision
models/gemini-1.0-pro-vision-latest
The best image understanding model to handle a broad range of applications

9 )   Gemini 1.0 Pro Vision
models/gemini

In [18]:
for model in genai.list_models():
  if model.name == 'models/gemini-1.5-flash':
    print(model)
    break
      

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)


## Exploring parameter

### Output length:

| max_output_length

irrespective of how much response is generated this parameters determines the length of printed response

In [19]:
model_1 = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(max_output_tokens=200)
)

Markdown(model_1.generate_content('Write a 1000 word essay on Yourself, be a narcissist').text)

## A Masterpiece in the Making: The Untold Story of My Extraordinary Self

I've always been a bit different. Not in a quirky, endearing way, mind you. No, I'm different in a way that sets me apart. I'm not merely exceptional; I'm *extraordinary*. My mind operates on a plane most people can only dream of, fueled by a brilliance that burns brighter than any star. 

From a tender age, I was acutely aware of my unique qualities. While others were content with the mundane, I yearned for something more. I craved intellectual stimulation, the thrill of pushing boundaries, the intoxicating feeling of being understood, truly understood, by someone who could grasp the complexities of my mind. 

Sadly, I found such comprehension lacking in the world around me. School was a tedious formality, a mere stepping stone on my path to greatness. The teachers, well-meaning but ultimately limited, struggled to keep pace with my prodigious

In [20]:
print(len(model_1.generate_content('Write a 1000 word essay on Yourself, be a narcissist').text))

992


## Temperature

| temperature 

This control degree of randomness in token selection. 
Zero temperature results in greedy decoding, which selects most probable token at each step.

In [21]:
import time

In [22]:
high_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature = 2.0)
)



for _ in range(5):
  response = high_temp_model.generate_content('Pick a random colour... (answer in a single word)')
  if response.parts:
    print(response.text, '-' * 25)
    print(response.usage_metadata)

 

Green 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Blue 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14



In [23]:
low_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=0.0))

for _ in range(5):
  response = low_temp_model.generate_content('Pick a random colour... (answer in a single word)')
  if response.parts:
    print(response.text, '-' * 25)
    print(response.usage_metadata)

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14

Purple 
 -------------------------
prompt_token_count: 13
candidates_token_count: 1
total_token_count: 14



## top-K and top-p

| top-k :
defines number of most probable tokens to select output tokens from. basically giving token choices for output token. top-k = 1 creates greedy decoding.

| top-p
defines 'probability threshold' which determines the upper bound after which the token stops being selected as candidate.


In [24]:
model_2 = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=1.0,
        top_k=64,
        top_p=0.95,
    ))

In [25]:
story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = model_2.generate_content(story_prompt)
# print(response.text)
print(response.usage_metadata)
print((response.usage_metadata.candidates_token_count)/(response.usage_metadata.total_token_count))

prompt_token_count: 20
candidates_token_count: 621
total_token_count: 641

0.968798751950078


# Prompting

## Zero-Shot

directly describe the request

In [26]:
model_zs = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config= genai.GenerationConfig(
        temperature=1.0,
        top_p=1, #all tokens can become candidates
        max_output_tokens=5
    )
)

In [42]:
zs_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """


Markdown(model_zs.generate_content(zs_prompt).text)

Sentiment: **POSITIVE**

## Enum Mode

used to constrain the output of prompt to a fixed set of values


In [43]:
import enum 

In [51]:
class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


model_enum = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ))

response = model_enum.generate_content(zs_prompt)
print(response.text)

positive


## One-shot and Few-shot

1. Provide example
2. Multiple examples

In [52]:
model_eg = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ))

In [53]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "peperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}

ORDER:
"""


customer_order = "Give me a large with cheese and pineapple"

Markdown(model_eg.generate_content([few_shot_prompt,customer_order]).text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
``` 


## JSON Mode


In [54]:
import typing_extensions as typing

class pizza_order(typing.TypedDict):
    size : str
    ingredients: list[str]
    type:str


model_pz = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=pizza_order,
    ))

print(model_pz.generate_content("Can I have a large dessert pizza with apple and chocolate").text)

    

{"ingredients": ["apple", "chocolate"], "size": "large", "type": "dessert"}

